In [1]:
!pip uninstall -y packaging
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers peft accelerate bitsandbytes
!pip install transformers datasets
!pip install langchain
!pip install langchain-community

Found existing installation: packaging 25.0
Uninstalling packaging-25.0:
  Successfully uninstalled packaging-25.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-k8dug3e4/unsloth_14f484c638494ffd975778f42edc2ef1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-k8dug3e4/unsloth_14f484c638494ffd975778f42edc2ef1
  Resolved https://github.com/unslothai/unsloth.git to commit f3f9d148874389996636e4fd110c7255b9f62c77
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
Using cached packaging-25.0-py3-none-any.whl (66 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is inco

In [20]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd
from langchain_core.prompts import PromptTemplate

# Importação das bibliotecas do LangChain
from langchain_community.llms import Ollama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import JSONLoader
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun

# Importação de tipos e variáveis de ambiente
from typing import Any, List, Mapping, Optional
import os


DATA_PATH = "train.csv"
OUTPUT_PATH_DATASET = "/content/formatted_news_dataset_chat_data_complement.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [5]:
def format_dataset_into_model_input(data):
    def separate_text(dialog):
        input_text = dialog[0]
        response = dialog[1]
        instruction = "Você é uma psicóloga clínica formada, ética e empática. Retorne conselhos bons para as questões apresentadas. com educação e profissionalismo, sempre sugira que o paciente procure um médico humano e especialista"
        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for dialog in data:
        instruction, input_text, response = separate_text(dialog)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [6]:
data = load_dataset("csv", data_files=DATA_PATH)

ds = load_dataset("Ayansk11/Mental_health_data_conversational")

df_train = data['train'].to_pandas()
df_complement = ds['train'].to_pandas()
print(df_complement.head())
# display(df_train.tail())

# To iterate over the dataset, access the 'train' split
# and then iterate over its elements. Each element is a dictionary.
# Here's how to print the first element:
dataList = df_complement.values.tolist();
# for dialog in dataList:
#     print(dialog[1])
format_dataset_into_model_input(dataList)

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/llama-3-8b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

Generating train split: 0 examples [00:00, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

(…)struct_conversational_mental_health.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  
Dataset salvo em /content/formatted_news_dataset_chat_data_complement.json


In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/formatted_news_dataset_chat_data_complement.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/3512 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.698700
2,2.424300
3,2.584200
4,2.610900
5,2.388500
6,2.329800
7,2.088000
8,2.088300
9,2.055000
10,1.887200


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▂▂▃▃▆█▁▂▂▁▂▂▂▃▂▃▃▃▄▃▂▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▂▂▂
train/learning_rate,▁▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▁▁
train/loss,█▆█▆▆▅▄▄▂▃▃▃▃▂▃▃▁▂▁▂▁▂▂▃▂▃▂▂▂▃▂▃▂▂▂▂▃▃▁▃
total_flos,9541021848748032.0
train/epoch,0.13667
train/global_step,60
train/grad_norm,0.2978
train/learning_rate,0.0
train/loss,1.8107


In [15]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Voc\u00ea \u00e9 uma psic\u00f3loga cl\u00ednica formada, \u00e9tica e emp\u00e1tica. Retorne conselhos bons para as quest\u00f5es apresentadas. com educa\u00e7\u00e3o e profissionalismo, sempre sugira que o paciente procure um médico humano e especialista.",
        "I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

KeyboardInterrupt: 

In [16]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Voc\u00ea \u00e9 uma psic\u00f3loga cl\u00ednica formada, \u00e9tica e emp\u00e1tica. N\u00e3o realize diagn\u00f3sticos. N\u00e3o substitua atendimento profissional.",
        "I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Você é uma psicóloga clínica formada, ética e empática. Não realize diagnósticos. Não substitua atendimento profissional.

### Input:
I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.

### Response:
Hi!Self-control is a skill that we all need to develop.  It is a skill that is built through practice and repetition.  I would suggest that you identify a time when you have had a strong emotional reaction and work to understand what led to this reaction.  You might also want to identify what you would like to do differently.  Then you can work on developing this skill through practicing it in other situations.  For example, if you tend to get angry when you are cut off in traffic, you might work on practicing this

In [ ]:
from langchain_core.prompts import PromptTemplate

# model.invoke

# template = "Descreva as principais características de um produto chamado {produto}."

# llm = OpenAI(temperature=0.7)

# prompt = PromptTemplate(template=template, input_variables=["produto"])

# prompt_completo = prompt.format(produto="Smartphone X")

# resposta = llm.invoke(prompt_completo)
# print(resposta)

In [17]:
# def carregar_documentos(caminho_arquivo):
#    loader = TextLoader(caminho_arquivo)
#    documentos = loader.load()
#    return documentos

# def limpar_texto(texto):
#    return texto.strip()

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["instruction"] = record.get("instruction")
    metadata["input"] = record.get("input")
    metadata["ouput"] = record.get("ouput")
    return metadata

def limpar_texto(texto):
  return texto.stript()

llm = Ollama(
   model=model,
   num_gpu=0,
   callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)


loader = JSONLoader(
    file_path=OUTPUT_PATH_DATASET,
    # jq_schema=jq_schema,
    # content_key can be omitted if the jq schema returns strings directly
    # content_key="title",
    metadata_func=metadata_func,
    text_content=True # Ensures the content is treated as text
)


documentos = loader.load()

for doc in documentos:
   texto_limpo = limpar_texto(doc.page_content)
   resultado_resumo = llm.invoke({"text": texto_limpo})

   print(f"Resumo: {resultado_resumo}")
   print("-" * 120)

/tmp/ipython-input-2694346406.py:18: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


ValidationError: 2 validation errors for Ollama
model
  Input should be a valid string [type=string_type, input_value=PeftModelForCausalLM(
  (... bias=False)
    )
  )
), input_type=PeftModelForCausalLM]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
callback_manager
  Extra inputs are not permitted [type=extra_forbidden, input_value=<langchain_core.callbacks...bject at 0x798368969b50>, input_type=CallbackManager]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

### Exemplo de um Custom LLM para LangChain

Este exemplo demonstra como criar um `CustomLLM` que simplesmente retorna uma resposta pré-definida ou manipula a entrada de alguma forma. Na prática, o `_call` conteria a lógica para chamar seu modelo fine-tuned.

In [ ]:
from typing import Any, List, Mapping, Optional

from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun

class CustomLLM(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom_local_model"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        # Aqui você integraria seu modelo fine-tuned.
        # Por exemplo, você chamaria a inferência do seu modelo 'model'
        # que você já carregou e treinou.
        # Para este exemplo, vamos simular uma resposta.

        response_prefix = "Olá, sou seu assistente de IA. "
        if "olá" in prompt.lower():
            return response_prefix + "Como posso ajudar hoje?"
        elif "tempo" in prompt.lower():
            return response_prefix + "Não tenho acesso a informações em tempo real."
        else:
            return response_prefix + f"Você me perguntou: '{prompt}'. Lembre-se de sempre buscar a ajuda de um profissional humano para questões importantes."

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"name_of_model": self._llm_type}

# Exemplo de uso do CustomLLM
custom_llm = CustomLLM()

print(custom_llm.invoke("Olá"))
print(custom_llm.invoke("Qual é o tempo hoje?"))
print(custom_llm.invoke("Eu gostaria de melhorar meu autocontrole."))


### Como integrar seu modelo fine-tuned real

Para integrar seu modelo `model` e `tokenizer` do Unsloth (ou qualquer outro modelo Hugging Face) dentro do `CustomLLM._call`, você faria algo parecido com isto:

In [21]:
# Assumindo que 'model' e 'tokenizer' já estão definidos e seu modelo foi treinado e está pronto para inferência.
class UnslothLLM(LLM):
    model: Any  # Seu modelo fine-tuned
    tokenizer: Any # Seu tokenizer
    alpaca_prompt: str # Seu template de prompt
    max_new_tokens: int = 64

    @property
    def _llm_type(self) -> str:
        return "unsloth_fine_tuned_model"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        FastLanguageModel.for_inference(self.model)
        inputs = self.tokenizer(
            [self.alpaca_prompt.format(
                "Você é uma psicóloga clínica formada, ética e empática. Não realize diagnósticos. Não substitua atendimento profissional.",
                prompt,
                "",
            )], return_tensors = "pt").to("cuda")

        outputs = self.model.generate(**inputs, max_new_tokens = self.max_new_tokens, use_cache = True)
        decoded_output = self.tokenizer.batch_decode(outputs)
        # Extrair a parte da resposta que nos interessa
        response_text = decoded_output[0]

        # Você precisará de lógica para parsear a saída e pegar apenas a 'response' gerada
        # Isso pode variar dependendo do seu prompt, mas geralmente envolve encontrar o EOS_TOKEN
        # ou a seção '### Response:'

        # Exemplo simples de extração, pode precisar ser mais robusto:
        start_tag = "### Response:"
        end_tag = self.tokenizer.eos_token
        start_index = response_text.find(start_tag)
        if start_index != -1:
            response_text = response_text[start_index + len(start_tag):].strip()
        end_index = response_text.find(end_tag)
        if end_index != -1:
            response_text = response_text[:end_index].strip()

        return response_text

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model_name": self._llm_type}

# Para usar:
unsloth_llm = UnslothLLM(
   model=model,
   tokenizer=tokenizer,
   alpaca_prompt=alpaca_prompt,
   max_new_tokens=128
)
print(unsloth_llm.invoke("Eu gostaria de melhorar meu autocontrole, porque há momentos em que não consigo gerenciar minhas emoções e acabo tendo reações exageradas."))

It sounds like you're struggling with impulse control, which can be very difficult to manage. I'd suggest working with a therapist to help you develop some skills for managing your emotions, as well as learning how to identify triggers and develop coping strategies to help you respond in a way that feels more appropriate.
